In [4]:
import matplotlib.pyplot as plt

from pydub import AudioSegment

from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter

from pyAudioAnalysis import audioSegmentation as aS

import moviepy.editor as mp

%matplotlib inline

## Get Audio Segments from Video
Uses MoviePy to grab audio in ~9min increments from given video

In [5]:
video_path = "../voices/baltimore_video.mp4"
audio_path = ""
audio_out_path = "../voices/audio/s08e15_baltimore_{}.mp3"

In [14]:
60*45

2700

In [15]:
video_input = mp.VideoFileClip(video_path)

timestamps = [[540*x, 540*(x+1)] for x in range(5)]
timestamps[-1][1] = video_input.duration

In [16]:
clips = [video_input.subclip(stamp[0], stamp[1]) for stamp in timestamps]
clips[0]

In [18]:
for n in range(5):
    clip = clips[n]
    clip.audio.write_audiofile(audio_out_path.format(n))

chunk:   2%|▏         | 234/11908 [00:00<00:05, 2309.06it/s, now=None]

MoviePy - Writing audio in ../voices/audio/s08e15_baltimore_0.mp3



chunk:   1%|          | 104/11908 [00:00<00:11, 1027.34it/s, now=None]

MoviePy - Done.
MoviePy - Writing audio in ../voices/audio/s08e15_baltimore_1.mp3



chunk:   1%|          | 103/11908 [00:00<00:12, 958.50it/s, now=None]

...

MoviePy - Done.
MoviePy - Writing audio in ../voices/audio/s08e15_baltimore_3.mp3



chunk:   1%|▏         | 138/10399 [00:00<00:07, 1360.73it/s, now=None]

MoviePy - Done.
MoviePy - Writing audio in ../voices/audio/s08e15_baltimore_4.mp3



chunk:   0%|          | 35/11908 [01:03<5:57:19,  1.81s/it, now=None]   

MoviePy - Done.


## Separate background noise & dialogue
Uses Spleeter 2 stem model.

In [20]:
# Using embedded configuration.
separator = Separator('spleeter:2stems')

In [21]:
for n in range(5):
    separator.separate_to_file(audio_out_path.format(n), '../voices/spleeter')
    print('Finished spleeting file {}'.format(n))

INFO:tensorflow:Apply unet for vocals_spectrogram


Exception ignored in: <function FFMPEG_AudioWriter.__del__ at 0x7fee7fec2dc0>
Traceback (most recent call last):
  File "/Users/mirielle/.local/share/virtualenvs/extract-tts-data-7AsNQbjF/lib/python3.8/site-packages/moviepy/audio/io/ffmpeg_audiowriter.py", line 132, in __del__
    self.close()
  File "/Users/mirielle/.local/share/virtualenvs/extract-tts-data-7AsNQbjF/lib/python3.8/site-packages/moviepy/audio/io/ffmpeg_audiowriter.py", line 121, in close
    self.proc.stdin.close()
BrokenPipeError: [Errno 32] Broken pipe


INFO:tensorflow:Apply unet for accompaniment_spectrogram
INFO:tensorflow:Restoring parameters from pretrained_models/2stems/model


In [11]:
spleeter_file_format = '../voices/spleeter/s08e15_baltimore_{}/vocals.wav'
for n in range(5):
    filepath = spleeter_file_format.format(n)
    audio_clip = AudioSegment.from_mp3(filepath)
    audio_clip = audio_clip.set_frame_rate(16000)
    mono_audio = audio_clip.set_channels(1)
    mono_audio.export(filepath, format="wav")
    print("Exported mono audio with sample rate 16000 to {}".format(filepath))

Exported mono audio to ../voices/spleeter/s08e15_baltimore_0/vocals.wav
Exported mono audio to ../voices/spleeter/s08e15_baltimore_1/vocals.wav
Exported mono audio to ../voices/spleeter/s08e15_baltimore_2/vocals.wav
Exported mono audio to ../voices/spleeter/s08e15_baltimore_3/vocals.wav
Exported mono audio to ../voices/spleeter/s08e15_baltimore_4/vocals.wav


## Speaker Diarization
Identify the different speakers using pyAudioAnalysis

In [12]:
d_output = aS.speaker_diarization('../voices/spleeter/s08e15_baltimore_3/vocals.wav', 0)

ValueError: n_components cannot be larger than min(n_features, n_classes - 1).